In [6]:
import cv2
import numpy as np
import os
import scipy.io as sio
from extractors import Extractors
from random import randint
from matplotlib import pyplot as plt
from typing import Dict, List, Tuple

In [7]:
DATASET_FOLDER = "MSRC_ObjCategImageDatabase_v2"
OUT_FOLDER = "descriptors"
OUT_SUBFOLDER = 'globalRGBhisto'
DESCRIPTOR_FOLDER = 'descriptors'
DESCRIPTOR_SUBFOLDER = 'globalRGBhisto'
def create_descriptor_dir(out_folder, out_subfolder):
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
        os.makedirs(os.path.join(out_folder, out_subfolder), exist_ok=True)

# Extract Image Descriptors from the Dataset
def extract_descriptors(dataset_path, descriptor_path, extract_method=Extractors.extract_rgb):
   for filename in os.listdir(os.path.join(dataset_path, 'Images')):
    if filename.endswith(".bmp"):
        img_path = os.path.join(dataset_path, 'Images', filename)
        img = cv2.imread(img_path).astype(np.float64) / 255.0  # Normalize the image
        fout = os.path.join(descriptor_path, filename).replace('.bmp', '.npy')
        F = extract_method(img)        
        np.save(fout, F)

create_descriptor_dir(OUT_FOLDER, OUT_SUBFOLDER)
extract_descriptors(DATASET_FOLDER, os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER))

In [8]:
def get_image_descriptor_mapping(descriptor_type='globalRGBhisto') -> Dict[str, np.ndarray]:
    descriptor_path = os.path.join(OUT_FOLDER, OUT_SUBFOLDER)
    img_to_descriptor = {}

    for filename in os.listdir(descriptor_path):
        if filename.endswith('.npy'):
            img_path = os.path.join(DATASET_FOLDER, 'Images', filename.replace('.npy', '.bmp'))
            descriptor_data = np.load(os.path.join(descriptor_path, filename))
            img_to_descriptor[img_path] = descriptor_data

    return img_to_descriptor

img2descriptors = get_image_descriptor_mapping()

In [10]:
def cvpr_compare(F1, F2):
    # This function should compare F1 to F2 - i.e. compute the distance
    # between the two descriptors
    # For now it just returns a random number
    # dst = np.random.rand()
    dst = np.linalg.norm(F1-F2)
    return dst

def compute_distance(query_img: str, img_desc_dict: Dict[str, np.ndarray]) -> List[Tuple[float, str]]:
    # Compute the distance between the query and all other descriptors
    dst = []
    query_img_desc = img_desc_dict[query_img]
    
    for img_path, candidate_desc in img_desc_dict.items():
        if img_path != query_img:  # Skip the query image itself
            distance = cvpr_compare(query_img_desc, candidate_desc)
            dst.append((distance, img_path))
    
    dst.sort(key=lambda x: x[0])
    return dst

distances = compute_distance("MSRC_ObjCategImageDatabase_v2/Images/16_22_s.bmp", img2descriptors)
distances

[(np.float64(0.011929565927703455),
  'MSRC_ObjCategImageDatabase_v2/Images/17_5_s.bmp'),
 (np.float64(0.013917114987563766),
  'MSRC_ObjCategImageDatabase_v2/Images/8_4_s.bmp'),
 (np.float64(0.017697364694640796),
  'MSRC_ObjCategImageDatabase_v2/Images/3_29_s.bmp'),
 (np.float64(0.01979484027715839),
  'MSRC_ObjCategImageDatabase_v2/Images/17_7_s.bmp'),
 (np.float64(0.020235902442524472),
  'MSRC_ObjCategImageDatabase_v2/Images/2_9_s.bmp'),
 (np.float64(0.0227513185377321),
  'MSRC_ObjCategImageDatabase_v2/Images/4_24_s.bmp'),
 (np.float64(0.02632903529154788),
  'MSRC_ObjCategImageDatabase_v2/Images/8_20_s.bmp'),
 (np.float64(0.028328333507613466),
  'MSRC_ObjCategImageDatabase_v2/Images/4_19_s.bmp'),
 (np.float64(0.028360303652696508),
  'MSRC_ObjCategImageDatabase_v2/Images/16_4_s.bmp'),
 (np.float64(0.030106246089515176),
  'MSRC_ObjCategImageDatabase_v2/Images/8_1_s.bmp'),
 (np.float64(0.030974542478379465),
  'MSRC_ObjCategImageDatabase_v2/Images/2_10_s.bmp'),
 (np.float64(0.03

In [11]:
def retrieve_similar_images(query_img, number=15) -> list:
    res = []
    return res

def display_images():
    return

In [6]:

# # Pick a random image as the query
# NIMG = ALLFEAT.shape[0]
# queryimg = randint(0, NIMG - 1)

# # Display the query image
# query_image_path = ALLFILES[queryimg]
# query_image = cv2.imread(query_image_path)

# if query_image is None:
#     print(f"Failed to load query image at {query_image_path}")
# else:
#     cv2.imshow("Query Image", query_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     cv2.waitKey(1)

# Show the top 15 results
SHOW = 15
images = [] 
target_height = 100

for i in range(min(SHOW, len(dst))):
    img_path = ALLFILES[dst[i][1]]
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Failed to load image at {img_path}")
        continue
    
    # Calculate the new width to maintain the aspect ratio
    aspect_ratio = img.shape[1] / img.shape[0]
    new_width = int(target_height * aspect_ratio)
    
    # Resize the image to the target height while maintaining the aspect ratio
    img = cv2.resize(img, (new_width, target_height))
    images.append(img)

# Concatenate images horizontally
if images:
    concatenated_image = np.hstack(images)
    cv2.imshow("Results", concatenated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
else:
    print("No images to display.")